In [1]:
from src.BinaryClassificationModel import BinaryClassificationModel
import pickle
import torch

with open('../data/processed/preprocessed.pkl', 'rb') as f:
    params = pickle.load(f)

model = BinaryClassificationModel(params['X_train'].shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [2]:
from src.binaryClassificationDataset import BinaryClassificationDataset
from torch.utils.data import DataLoader

dataset = BinaryClassificationDataset(list(zip(params['X_train'], params['y_train'])))
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

for epoch in range(30):
    model.train()
    total_loss = 0

    for X_batch, y_batch in dataloader:
        optimizer.zero_grad()
        y_batch = y_batch.unsqueeze(1)

        logits = model(X_batch)
        loss = model.loss(logits, y_batch)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader):.4f}")


Epoch 1, Loss: 0.4711
Epoch 2, Loss: 0.2039
Epoch 3, Loss: 0.1109
Epoch 4, Loss: 0.0850
Epoch 5, Loss: 0.0745
Epoch 6, Loss: 0.0692
Epoch 7, Loss: 0.0655
Epoch 8, Loss: 0.0631
Epoch 9, Loss: 0.0607
Epoch 10, Loss: 0.0590
Epoch 11, Loss: 0.0574
Epoch 12, Loss: 0.0560
Epoch 13, Loss: 0.0547
Epoch 14, Loss: 0.0535
Epoch 15, Loss: 0.0528
Epoch 16, Loss: 0.0519
Epoch 17, Loss: 0.0512
Epoch 18, Loss: 0.0504
Epoch 19, Loss: 0.0497
Epoch 20, Loss: 0.0492
Epoch 21, Loss: 0.0486
Epoch 22, Loss: 0.0476
Epoch 23, Loss: 0.0471
Epoch 24, Loss: 0.0467
Epoch 25, Loss: 0.0460
Epoch 26, Loss: 0.0457
Epoch 27, Loss: 0.0452
Epoch 28, Loss: 0.0448
Epoch 29, Loss: 0.0442
Epoch 30, Loss: 0.0436


In [6]:
test_set = BinaryClassificationDataset(list(zip(params['X_test'], params['y_test'])))
test_loader = DataLoader(test_set, batch_size=10, shuffle=True)

model.eval()
test_loss = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        y_batch = y_batch.unsqueeze(1)
        logits = model(X_batch)
        loss = model.loss(logits, y_batch)
        test_loss += loss.item()

print(f"Test Loss: {test_loss/len(test_loader):.4f}")

Test Loss: 0.0745


In [5]:
torch.save(model.state_dict(), "../models/BinaryClassificationModel.pth")